# Import Library

In [1]:
import pandas as pd
import numpy as np
#Import Library untuk Tokenisasi
import string 
import re #regex library

# import word_tokenize & FreqDist dari NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Read Data PTA

In [2]:
dataPTA = pd.read_excel('PTAscrawl.xlsx')

In [3]:
dataPTA.head(10)

,Judul,Penulis,Dosen 1,Dosen 2,Abstrak
0,SISTEM PENDUKUNG KEPUTUSAN PEMILIHAN KARYAWAN ...,Penulis : Catur Ngesti Waluyo,"Dosen Pembimbing I : Mula'ab,S.Si.,M.Kom","Dosen Pembimbing II :Riza Alfita, S.T., M.T",Sumber daya manusia mutlak dibutuhkan untuk ke...
1,Perancangan Sistem Informasi Badan Kepegawaian...,Penulis : MALIKUL HAMZAH,"Dosen Pembimbing I : Moch. Kautsar Sophan, S.K...","Dosen Pembimbing II :Yeni Kustiyaningsih, S.Ko...",Kantor Badan Kepegawaian kota Bangkalan adalah...
2,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST ...,Penulis : M Khoiril Anwar,"Dosen Pembimbing I : Cucun Very Angkoso, S.T.,...","Dosen Pembimbing II :Arik Kurniawati S. Kom., ...",Teknologi mobile game beroperating system open...
3,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,Penulis : Muhammad Choirur Rozi,"Dosen Pembimbing I : Dr. Arif Muntasa, S.Si.,M.T","Dosen Pembimbing II :Fitri Damayanti, S.Kom.,M...",Sistem pengenalan wajah adalah suatu sistem un...
4,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,Penulis : NURRACHMAT,"Dosen Pembimbing I : Arik Kurniawati, S.Kom., ...","Dosen Pembimbing II :Kurniawan Eka Permana, S....","Perkembangan game yang semakin pesat, memberik..."
5,AUTO LEVELING BERBASIS FINITE STATE MACHINE (F...,Penulis : Syafrial Nur Maulana,"Dosen Pembimbing I : Arik Kurniawati, S.Kom.,M.T.","Dosen Pembimbing II :Ari Kusumaningsih, S.T., ...",Pengetahuan tentang bahasa Inggris pada jaman ...
6,Gerak Pekerja Pada Game Real Time Strategy Men...,Penulis : Adi Chandra Laksono,"Dosen Pembimbing I : Kurniawan Eka P, S.Kom., Msc","Dosen Pembimbing II :Arik Kurniawati, S.Kom., ...",Gerak pekerja ada pada game yang memiliki genr...
7,PENGEMBANGAN MESIN PENCARIAN ANTIPLAGIASI PADA...,Penulis : ilham wibisono aziz,"Dosen Pembimbing I : Hermawan S.T.,M.Kom",Dosen Pembimbing II :Andharini Dwi Cahyani S.K...,ABSTRAK_x000D_\nPenjiplakan merupakan masalah ...
8,ANALISA VALIDITAS PENERIMA BEASISWA MENGGUNAKA...,Penulis : Hilmi Fairuz Abadi,"Dosen Pembimbing I : Bain Khusnul K., S.T., M....","Dosen Pembimbing II :Firli Irhamni, S.T., M.Kom.",Beasiswa adalah pemberian berupa bantuan keuan...
9,SISTEM PERAMALAN PENJUALAN JANGKA PENDEK SPARE...,Penulis : Ana Qaimah L.M.D.R,"Dosen Pembimbing I : Mula’ab, S.Si, M.Kom","Dosen Pembimbing II :Fika Hastarita R.,ST, M.Eng",Spare part merupakan salah satu bagian penting...


# Case Folding

In [4]:

# gunakan fungsi Series.str.lower() pada Pandas
dataPTA['Abstrak'] = dataPTA['Abstrak'].str.lower()

print('Case Folding Result : \n')

#cek hasil case fold
print(dataPTA['Abstrak'].head(5))
print('\n\n\n')

Case Folding Result : 

0    sumber daya manusia mutlak dibutuhkan untuk ke...
1    kantor badan kepegawaian kota bangkalan adalah...
2    teknologi mobile game beroperating system open...
3    sistem pengenalan wajah adalah suatu sistem un...
4    perkembangan game yang semakin pesat, memberik...
Name: Abstrak, dtype: object






# Removal

In [5]:
def remove_PTA_special(text):
    # menghapus tab, new line, dan back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # menghapus non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # menghapus mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # menghapus incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
dataPTA['Abstrak'] = dataPTA['Abstrak'].apply(remove_PTA_special)

#menghapus nomor
def remove_number(text):
    return  re.sub(r"\d+", "", text)

dataPTA['Abstrak'] = dataPTA['Abstrak'].apply(remove_number)

#menghapus punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

dataPTA['Abstrak'] = dataPTA['Abstrak'].apply(remove_punctuation)

#menghapus spasi leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

dataPTA['Abstrak'] = dataPTA['Abstrak'].apply(remove_whitespace_LT)

#menghapus spasi tunggal dan ganda
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

dataPTA['Abstrak'] = dataPTA['Abstrak'].apply(remove_whitespace_multiple)

# menghapus kata 1 abjad
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

dataPTA['Abstrak'] = dataPTA['Abstrak'].apply(remove_singl_char)

print('Result : \n') 
print(dataPTA['Abstrak'].head())
print('\n\n\n')

Result : 

0    sumber daya manusia mutlak dibutuhkan untuk ke...
1    kantor badan kepegawaian kota bangkalan adalah...
2    teknologi mobile game beroperating system open...
3    sistem pengenalan wajah adalah suatu sistem un...
4    perkembangan game yang semakin pesat memberika...
Name: Abstrak, dtype: object






# STOPWORD 

In [6]:
list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(["aam","absolute","abstract","abstrakxd","adm","ahp","ai","aid","akanxd","akhirxd","alert","algorithm",
                       "alpha","alternative","ambroxol","analysis","analytic","analytical","and","angkaangka","angular","anp",
                       "apl","aplikasixd","application","architecture","artifical","as","asesoris","attribute","automatic",
                       "average","babnyajika","background","bahanbahan","baikxd","balanced","base","based","basic","bc",
                       "beasiwa","benarbenar","benedict","beratxd","berbedabeda","berturutturut","bifurcation","binaryzation",
                       "bisoprolol","bkd","block","blue","bold","bolditalic","bpp","bps","browsing","bsc","business","by",
                       "canny","caps","cbir","cefixime","center","centroid","chain","chaining","chainning","character",
                       "cipherteks","class","classfier","classification","classifier","close","cluster","clustering","coding",
                       "combat","commerce","component","compute","computer","confix","content","contex","context","corepoint",
                       "corpus","cosine","criteria","criteriaxd","crm","crossing","customer","cut","cycle","dalamxd","darixd",
                       "database","datadata","dataxd","daviesbouldin","decision","decomposition","defuzzyfikasi","dekripsi",
                       "denga","denganxd","depanxd","depth","design","development","dibatasixd","dicarixd","difference",
                       "diprosesxd","direction","disegmentasi","disk","disperindag","distance","distemming",
                       "dkpp","dlda","dominant","download","dperoleh","dr","drop","dsebut","eclipse","ecommerce","ecommercexd",
                       "economic","edge","edm","education","egovernment","eigenxd","ekuitas","electre","electreii","electronic",
                       "emulator","engghibunten","engine","engineering","enginexd","english","enhanced","enjeiyeh","enterprise",
                       "environment","eoq","epoch","epoh","error","eucledian","euclidean","exploiting","exponential","express",
                       "fahp","fanp","feature","fighting","filter","filtering","fine","fingerprint","fingerprintbitmaps",
                       "finite","firewall","first","fisik","fmeasure","fmop","font","foreign","forward","framework","free",
                       "frustasi","fsm","function","fuzzy","fvc","galis","game","games","garisgaris","gateway","gaussian",
                       "geometry","gizixd","gldm","glrm","gr","gradient","gradients","gray","grayscaling","grcitra","ground",
                       "growth","ha","haar","had","handwriting","harris","hash","hh","hidden","hierarchy","high",
                       "hijauxd","hl","hog","ht","idb","ii","ij","iksass","image","indahmulya","indicator","indicators",
                       "infoinfo","inginxd","inixd","inktech","interface","interprise","intervace","intervensi","interview",
                       "intraseluler","intrusion","invariant","inventori","ips","iptables","italic",
                       "jaringanjaringan","jarixd","java","jejaring","jiwaxd","jst","kabupatenkabupaten","kaganga","kallista",
                       "karakterkarakter","karapanxd","kec","kerapan","kerjasamaxd","kesejahteraanya","key","keypoint",
                       "keyword","keywords","kg","kit","kkm","kluster","kmeans","kohonen","kokop","komulatif","konang",
                       "konekasi","kpi","kriteriakriteria","kriteriaxd","ksom","kub","kuisioner","kuisoner","lainlain","lainxd",
                       "langkahlangkah","language","languange","latent","layer","lda","learning","least","length","lerning",
                       "leveling","lh","life","light","linier","link","listening","ll","load","log","logic","low","lsa",
                       "lsasom","lt","lunakxd","lunturnya","lvq","lyapunov","machine","madistrindo","maduraindonesia",
                       "maduraxd","mail","making","malan","mamdani","management","manager","mandiriauto","map","mape","maps",
                       "mapserver","martodirdjo","masingmasing","masking","matching","matrix","maze","mazexd","mcdm","mdf",
                       "mean","melakukanxd","membatu","memilikixd","mengenkripsi","menggunakanxd","message",
                       "metadata","method","mg","middleware","minimnya","minutea","minutiae","modung",
                       "momentum","monitoring","morfologi","mosaic","mosaikpanoramik","moving","mpc","mse","multiatribute",
                       "multimedia","multiobjective","multiple","naive","nave","nbc","negaranegara","network","neural","ngram",
                       "node","nomor","non","npc","number","numberxd","nya","obatobatan","objective","obyek","obyektif","of",
                       "offline","ofr","oldinary","ols","omax","ontologi","ontology","open","optical","optimized","optimiztion",
                       "optimum","ordered","organizing","oriented","orl","output","owl","panoramic","panoramik","panoteng",
                       "parameterparameter","parsing","part","particle","pasienxd","pattern","pca","pe","pejualan",
                       "pelajarsantri","pelevelan","pemvalidasian",
                       "penjadwaln","perankingan","perankingannya","percentage","perconbaan","performance",
                       "periodeperiode","permasalaha","perusahaanxd","pihakpihak","pihakxd","pixel","pixels","plainteks",
                       "plasmodium","plastec","platform","playable","player","pmg","podhek","point","pose","ppa","prakandidat",
                       "precision","preference","presentase","preshion","prevention","prim","principal","prinsipnya","print",
                       "prism","probabilitasmetode","process","processing","produksipada","produktivitas","profitabilitas",
                       "programing","programming","programprogram","project","prosentase","prosesnya","pso","pt","ptxd",
                       "quantity","quantization","query","rangkebbhan","rank","ranks","raskin","ratarata","rate","rater",
                       "rating","ratus","rbfn","rbfnn","rbfnnxd","rc","rdf","reading","real","realistisxd","realitas","reality",
                       "realtime","recall","recognition","rekomndasi","relative","release","resource","resources","responden",
                       "retrieval","reuse","ridge","riilxd","rill","riwayatxd","rehabilitasi","roughness","rts","run","saaty",
                       "salafiyah","roughness","sales","sasaranxd","satunya","satunya","scale","scm","scorecard","scoring",
                       "screen","sdk","sdkxd","sdlc","sdm","search","second","security","segmentasinya","seharihari",
                       "sekuensial","self","semantic","sencitivity","seolaholah","separation","seringkali","server","service",
                       "ses","seseorangxd","shop","shortest","sift","sikannya","similar","similaritas","similarity","simple",
                       "simtak","single","singular","sistemxd","skenarioskenario","sky","sma","smarter","smartphone",
                       "smoothing","smooting","smp","sms","snort","software","solusinya","solusinyaxd","solution","som","sort",
                       "source","spare","spasial","spci","speaking","specificity","speech","spk","square","stakeholder","state",
                       "statistik","statusxd","stemmer","stemming","stockpile","strategi","strategy","straw","stripping",
                       "style","sub","subkriteria","subset","subsistem","subtropics","subyektivitas","sumenep","suplier",
                       "supplier","supply","swarm","syafiiyah","system","tab","tamansepanjang","technique","telekomunikasi",
                       "terater","terhadapxd","termination","terpisahpisah","tersebutxd","tertentuumumnya","test","testes",
                       "testing","thinning","thomas","threshold","tiaptiap","time","tinggixd","titiktitik","tnpk","to","toba",
                       "toefl","toeflxd","togaf","tool","tools","tooltool","topsis","traffic","tragah","training","transform",
                       "treshold","truth","tsai","tujuansetelah","tulangan","tuneup","two","ujicoba","userxd","utnuk",
                       "validitas","value","vector","velocity","vii","virtual","vision","vr","wachid","waktuxd","waterfall",
                       "watershed","wavelet","web","website","webxd","wide","window","winnowing","world","www","xd","xna",
                       "yakersuda","yangxd",'baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'])
# Mengubah List ke dictionary
list_stopwords = set(list_stopwords)

In [7]:
dataPTA.head()

,Judul,Penulis,Dosen 1,Dosen 2,Abstrak
0,SISTEM PENDUKUNG KEPUTUSAN PEMILIHAN KARYAWAN ...,Penulis : Catur Ngesti Waluyo,"Dosen Pembimbing I : Mula'ab,S.Si.,M.Kom","Dosen Pembimbing II :Riza Alfita, S.T., M.T",sumber daya manusia mutlak dibutuhkan untuk ke...
1,Perancangan Sistem Informasi Badan Kepegawaian...,Penulis : MALIKUL HAMZAH,"Dosen Pembimbing I : Moch. Kautsar Sophan, S.K...","Dosen Pembimbing II :Yeni Kustiyaningsih, S.Ko...",kantor badan kepegawaian kota bangkalan adalah...
2,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST ...,Penulis : M Khoiril Anwar,"Dosen Pembimbing I : Cucun Very Angkoso, S.T.,...","Dosen Pembimbing II :Arik Kurniawati S. Kom., ...",teknologi mobile game beroperating system open...
3,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,Penulis : Muhammad Choirur Rozi,"Dosen Pembimbing I : Dr. Arif Muntasa, S.Si.,M.T","Dosen Pembimbing II :Fitri Damayanti, S.Kom.,M...",sistem pengenalan wajah adalah suatu sistem un...
4,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,Penulis : NURRACHMAT,"Dosen Pembimbing I : Arik Kurniawati, S.Kom., ...","Dosen Pembimbing II :Kurniawan Eka Permana, S....",perkembangan game yang semakin pesat memberika...


In [8]:
vect =TfidfVectorizer(stop_words=list_stopwords,max_features=1000) 
vect_text=vect.fit_transform(dataPTA['Abstrak'])
print(vect_text.shape)
print(vect_text)


(65, 1000)
  (0, 376)	0.08161231722757978
  (0, 489)	0.08161231722757978
  (0, 464)	0.06778470526065353
  (0, 69)	0.08970095170275473
  (0, 43)	0.08970095170275473
  (0, 252)	0.04586845881855231
  (0, 356)	0.0758733397358285
  (0, 750)	0.06778470526065353
  (0, 906)	0.05236031914022402
  (0, 551)	0.03502385948167356
  (0, 667)	0.08970095170275473
  (0, 207)	0.08970095170275473
  (0, 305)	0.07527740138862064
  (0, 754)	0.08970095170275473
  (0, 93)	0.08970095170275473
  (0, 798)	0.03318110160471517
  (0, 140)	0.08970095170275473
  (0, 167)	0.11518846198309596
  (0, 402)	0.062045727768902235
  (0, 835)	0.04700871357164142
  (0, 416)	0.0758733397358285
  (0, 431)	0.20352774874840684
  (0, 112)	0.04586845881855231
  (0, 659)	0.3235477457701398
  (0, 760)	0.061924516766453906
  :	:
  (64, 490)	0.1382721204366279
  (64, 105)	0.19084364493417733
  (64, 860)	0.22309631148624776
  (64, 714)	0.26031936133942574
  (64, 483)	0.06176566737536445
  (64, 577)	0.07436543716208259
  (64, 707)	0.0650798

In [18]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)

In [19]:
print(lsa_top)
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

[[ 2.17153509e-01 -3.08812720e-01 -2.68997300e-01 -9.64533429e-02
  -1.32262669e-01 -2.30130439e-01 -5.51614601e-02 -4.01818854e-01
   9.03859182e-02  9.27746078e-02]
 [ 1.70104353e-01 -1.71894081e-01  1.44405702e-01 -2.38190627e-02
  -2.83579273e-01  7.22240010e-01  1.92842256e-02 -1.78317869e-01
  -1.16997367e-01 -3.57561501e-03]
 [ 1.51956943e-01 -1.69645126e-01  4.90535975e-01  5.62096748e-02
   1.46206848e-01 -1.19630890e-01  4.57444196e-02 -7.03549751e-02
   1.74844492e-01 -2.79966390e-01]
 [ 2.71408736e-01  1.06909963e-01 -5.83759418e-02  2.04800541e-01
  -6.15110032e-03  2.86214480e-02  5.13345113e-02 -5.78692453e-02
   6.00444757e-02 -1.99936739e-03]
 [ 9.85170745e-02 -6.30063761e-02  1.86765031e-01  1.92818781e-02
   5.53986064e-02 -6.07889216e-02 -5.25963363e-02  8.44757768e-02
  -1.19496699e-01  2.12249604e-01]
 [ 1.48669427e-01 -1.43636004e-01  3.01455432e-01  7.57167388e-02
   3.95187074e-02 -5.12390792e-02  1.02082844e-02 -1.47194745e-01
   9.33849948e-02  1.48772472e-01

In [20]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  21.715350899583534
Topic  1  :  -30.881271957251705
Topic  2  :  -26.899730015696132
Topic  3  :  -9.645334291582582
Topic  4  :  -13.226266903206286
Topic  5  :  -23.013043863534556
Topic  6  :  -5.516146008259662
Topic  7  :  -40.18188537842842
Topic  8  :  9.038591820361244
Topic  9  :  9.277460778459842


In [21]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 1000)
[[ 0.01297081  0.00397193  0.00565642 ...  0.01397252  0.05445197
   0.03266684]
 [ 0.0076286  -0.00618351 -0.00943063 ...  0.01424513  0.04873721
   0.03516623]
 [-0.0066347   0.01715741 -0.0040804  ...  0.00250413  0.00566259
   0.00373151]
 ...
 [ 0.00081913 -0.01096063  0.01908439 ...  0.00162791  0.00521744
   0.0041085 ]
 [ 0.00985557  0.00738463  0.02304947 ... -0.00174446  0.00185695
  -0.00060862]
 [-0.01671668  0.01218407  0.02226098 ...  0.00478573 -0.01013601
   0.00571048]]


In [22]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
citra batik metode data proses sistem nilai hasil tekstur pengenalan 

Topic 1: 
citra batik tekstur ciri fitur kemiripan ekstraksi perolehan isi gambar 

Topic 2: 
bahasa algoritma madura mobile android teknologi pembelajaran aplikasi arsitektur pencarian 

Topic 3: 
tangan tulisan pengenalan sidik jari telapak skenario proses carakan senyum 

Topic 4: 
produksi peramalan perusahaan penjualan algoritma permintaan pelanggan penjadwalan bahasa komputer 

Topic 5: 
arsitektur bangkalan informasi dinas pelayanan bisnis tahapan peramalan kepegawaian sistem 

Topic 6: 
sidik jari pendeteksian citra manusia skenario gizi region titik pasien 

Topic 7: 
gizi pasien status peramalan obat balita penentuan data nilai kebutuhan 

Topic 8: 
gizi mobile pasien citra status android teknologi balita gerakan perusahaan 

Topic 9: 
gizi bahasa madura pasien pelanggan status batik perusahaan sidik jari 



C:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
